# Blight violations Analysis

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Read the data form .csv file
DataFrame = pd.DataFrame(pd.read_csv('Blight_Violations.csv', low_memory = False))

#Adding a compliance variable that is the predictive variable
DataFrame['compliance'] = np.zeros(len(DataFrame), dtype = int)

In [15]:
DataFrame.dtypes

Ticket ID                                 int64
Ticket Number                            object
Agency Name                              object
Inspector Name                           object
Violator Name                            object
Violator ID                               int64
Violation Street Number                   int64
Violation Street Name                    object
Violation Zip Code                       object
Mailing Address Street Number            object
Mailing Address Street Name              object
Mailing Address City                     object
Mailing Address State                    object
Mailing Address Zip Code                 object
Mailing Address Non-USA Code             object
Mailing Address Country                  object
Violation Date                           object
Ticket Issued Time                       object
Hearing Date                             object
Hearing Time                             object
Violation Code                          

### Data cleanup

In [21]:
# Generate a compliance variable that would be the predictive variable

def featureCleanup(DF):
    #drop the columns that do not provide additional information
    DF.drop(['Ticket Number'], axis = 1, inplace = True)
    DF.drop(['Violator Name'], axis = 1, inplace = True)
    DF.drop(['Violation Location'], axis =1, inplace = True)
    DF.drop(['Violation Parcel ID'], axis =1, inplace = True) 
    DF.drop(['Payment Date (Most Recent)'], axis = 1, inplace = True)
    # Drop the rows that have a disposition declaring not responsible for some reason
    DF.drop(DF.loc[DataFrame['Disposition'].str[:3]=='Not'].index, inplace = True)
    #Drop the rows that have Disposition PENDING
    DF.drop(DF.loc[DataFrame['Disposition']=='PENDING'].index, inplace = True)
    
    #Replace all NaN in each column with 0
    for col in DF.columns:
        if(DF[col].dtypes == 'object'):
            DF.loc[DF[col].isnull(),col] = '0.0'
        if(DF[col].dtypes == 'int64'):
            DF.loc[DF[col].isnull(),col] = 0            
        if(DF[col].dtypes == 'float64'):
            DF.loc[DF[col].isnull(),col] = 0       
           

    columnList = [item for item in DF.columns if(('Amount' in item)|('Fee' in item)|('Cost' in item)|('Balance' in item))]
    #convert dtypes appropirately  
    
    # Money columns
    for col in columnList:
        if(DF[col].dtypes == 'object'):
            print(col)
            DF[col] = DF.loc[DF[col].str[0]=='$',col].str[1:].astype(np.float64)
            
    DF.drop(DF.loc[DF['Violation Date'].str[-4:]=='3016'].index, inplace = True)
    DF['Violation Date'] = pd.to_datetime(DF['Violation Date'])
    DF['Violation Date'] = DF['Violation Date'] - DF['Violation Date'].min()
    
    return DF
    

### Populate Compliance variable (Predictive variable)

In [ ]:
def populateCompliance(DF):
    #if payment status is 'Paid in full' then compliance is 0
    DF.loc[DF['Collection Status']== 'In collections', 'compliance'] = 1
    DF.loc[DF['Payment Status']=='NO PAYMENT DUE' ,'compliance'] = 0    

In [22]:
DataFrameCpy = DataFrame.copy()
DF = featureCleanup(DataFrameCpy)
#populateCompliance(DF)


Fine Amount
Admin Fee
State Fee
Late Fee
Discount Amount
Clean Up Cost
Judgment Amount (Total Due)
Payment Amount (Sum of All Payments)
Balance Due


In [18]:
DF.dtypes

Ticket ID                                        int64
Agency Name                                     object
Inspector Name                                  object
Violator ID                                      int64
Violation Street Number                          int64
Violation Street Name                           object
Violation Zip Code                              object
Mailing Address Street Number                   object
Mailing Address Street Name                     object
Mailing Address City                            object
Mailing Address State                           object
Mailing Address Zip Code                        object
Mailing Address Non-USA Code                    object
Mailing Address Country                         object
Violation Date                          datetime64[ns]
Ticket Issued Time                              object
Hearing Date                                    object
Hearing Time                                    object
Violation 

In [63]:
DF['Violation Date'] = DF['Violation Date'] - DF['Violation Date'].min()

In [65]:
DF['Violation Date'].min()


Timedelta('0 days 00:00:00')

In [30]:
pd.to_datetime(DF['Payment Date (Most Recent)'][6])

Timestamp('2006-02-15 00:00:00')

In [32]:
len(DF.loc[DF['Payment Date (Most Recent)'] != '0.0'])

69050

In [5]:
print(np.unique(DataFrame['Disposition'])[:10])

[nan 'Not responsible by City Dismissal' 'Not responsible by Determination'
 'Not responsible by Dismissal' 'PENDING'
 'Responsible (Fine Waived) by Admission'
 'Responsible (Fine Waived) by City Dismissal'
 'Responsible (Fine Waived) by Determination'
 'Responsible - Compl/Adj by Default'
 'Responsible - Compl/Adj by Determination']


/usr/local/lib/python2.7/dist-packages/numpy/lib/arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))


In [7]:
len(DataFrame)-len(DataFrame.loc[DataFrame['Disposition'].str[:3]=='Not'])

246832

In [13]:
DataFrame.loc[DataFrame['Disposition'].str[:3]=='Not','Balance Due'].index[:1000]

Int64Index([   1,    3,    4,    5,    7,    9,   10,   12,   15,   16,
            ...
            9501, 9502, 9503, 9504, 9507, 9508, 9509, 9510, 9511, 9512],
           dtype='int64', length=1000)

In [20]:
len(DataFrame.loc[DataFrame['Disposition']=='PENDING'])

785